In [1]:
# Efficiencies to create input decks for:
efficiencies = [1, 0.8, 0.6, 0.4, 0.2, 0.15, 0.12, 0.10, 0.05, 0.0]

In [2]:
import glob, os, shutil
from itertools import islice
import numpy as np

# Setting Physical Constants

In [3]:
decay = input("Add decay to removal constant? ")

eff_strings = np.array(efficiencies) * 100
eff_strings = eff_strings.astype(int)
eff_strings = list(eff_strings.astype(str))
print("Efficiencies to create input decks: {}".format(eff_strings))

# If decay is yes, Xe-135 decay constant is added (not isotope depandant, yet)
xe_decay = 2.091667E-05 #[/s]

# Values from WorkBook All Lambda
MSBR_pre_eff = 0.009974174
MSRE_pre_eff = 0.002054191
#MSRE_pre_eff = 0.001580148
MSDR_pre_eff = 0.003975773

Add decay to removal constant? Y
Efficiencies to create input decks: ['100', '80', '60', '40', '20', '15', '12', '10', '5', '0']


In [4]:
# Empty list to add removal constants to
MSBR_r = []
MSRE_r = []
MSDR_r = []


print("Calculating Removal Constants based on efficiencies...")
if decay == "N":
    for i in efficiencies:
        removal_B = i * MSBR_pre_eff
        removal_E = i * MSRE_pre_eff
        removal_D = i * MSDR_pre_eff
        MSBR_r.append(removal_B)
        MSRE_r.append(removal_E)
        MSDR_r.append(removal_D)
elif decay == "Y":
    for i in efficiencies:
        removal_B = (i * MSBR_pre_eff) + xe_decay
        removal_E = (i * MSRE_pre_eff) + xe_decay
        removal_D = (i * MSDR_pre_eff) + xe_decay
        MSBR_r.append(removal_B)
        MSRE_r.append(removal_E)
        MSDR_r.append(removal_D)

Calculating Removal Constants based on efficiencies...


In [5]:
print(MSBR_r)
print(MSDR_r)
print(MSRE_r)

[0.00999509067, 0.00800025587, 0.00600542107, 0.004010586270000001, 0.0020157514700000004, 0.00151704277, 0.0012178175500000001, 0.0010183340700000002, 0.00051962537, 2.091667e-05]
[0.00399668967, 0.0032015350700000003, 0.00240638047, 0.0016112258700000002, 0.00081607127, 0.0006172826199999999, 0.00049800943, 0.00041849397, 0.00021970532000000002, 2.091667e-05]
[0.00207510767, 0.0016642694700000002, 0.00125343127, 0.00084259307, 0.00043175487, 0.00032904531999999995, 0.00026741958999999994, 0.00022633577000000002, 0.00012362622000000001, 2.091667e-05]


### This writes the input files:

In [6]:
# MSBR      
for i in range(0, len(MSBR_r)):
    with open('./MSBR_beta.txt', 'r') as file :
        filedata = file.read()
    
    # Replace the target string
    filedata = filedata.replace('substitute', str(MSBR_r[i]))
    file.close()
    
    # Write the file out again
    with open(('MSBR_beta_{}.inp'.format(eff_strings[i])), 'w') as file:
        file.write(filedata)
        file.close()
        
# MSRE      
for i in range(0, len(MSRE_r)):
    with open('./MSRE_beta.txt', 'r') as file :
        filedata = file.read()
    
    # Replace the target string
    filedata = filedata.replace('substitute', str(MSRE_r[i]))
    file.close()
    
    # Write the file out again
    with open(('MSRE_beta_{}.inp'.format(eff_strings[i])), 'w') as file:
        file.write(filedata)
        file.close()
        
# MSDR
for i in range(0, len(MSDR_r)):
    with open('./MSDR_beta.txt', 'r') as file :
        filedata = file.read()
    
    # Replace the target string
    filedata = filedata.replace('substitute', str(MSDR_r[i]))
    file.close()
    
    # Write the file out again
    with open(('MSDR_beta_{}.inp'.format(eff_strings[i])), 'w') as file:
        file.write(filedata)
        file.close()

Next, we create a folder for each input and place the input file inside:

In [7]:
# print(os.path.join('.', '*.inp*'))
# print(glob.glob(os.path.join('.', '*.inp*'))[0])
# print(glob.glob(os.path.join('.', '*.inp*'))[0].rsplit('.',1)[0])
# print(os.path.join('.', glob.glob(os.path.join('.', '*.inp*'))[0].rsplit('.',1)[0]))

# print(os.path.join(glob.glob(os.path.join('.', '*.inp*'))[0].rsplit('.',1)[0]), os.path.basename(os.path.join('.', '*.inp*'))[0])

In [8]:
# Creates folders based on input names and moves the input to the appropiate folder
folder = '.'

for file_path in glob.glob(os.path.join(folder, '*.inp*')):
    new_dir = file_path.rsplit('.', 1)[0]
    try:
        os.mkdir(os.path.join(folder, new_dir))
    except WindowsError:
        # Handle the case where the target dir already exist.
        pass
    shutil.move(file_path, os.path.join(new_dir, os.path.basename(file_path)))

In [9]:
# CREATES PBS FILE FOR EACH CASE AND PLACES IT IN THE FOLDER

# MSBR
# always remember, use files in a with statement
for i in range(0, len(eff_strings)):
    with open("scale_beta_unix.pbs", "r") as myfile: #JUST TO TAKE THE FIRST 23 LINES
        head = list(islice(myfile, 23))     
    with open("./MSBR_beta_{0}/MSBR_{0}.pbs".format(eff_strings[i]), "w", newline='\n') as f2:
        for item in head:
            f2.write(item)
            
        # SPECIFY PATH THAT THE PBS FILE WILL BE IN!
        f2.write("cd /home/ybb/SCALE_beta/MSBR/MSBR_beta_{}/\n".format(eff_strings[i]))
        f2.write("/scale/release/6.3b4/linux/bin/scalerte MSBR_beta_{0}.inp\n".format(eff_strings[i]))
        f2.write("\n")
        f2.close()
    myfile.close()
        
for i in range(0, len(eff_strings)):
    with open('./MSBR_beta_{0}/MSBR_{0}.pbs'.format(eff_strings[i]), 'r') as file :
        filedata = file.read()
        # Replace the target string
        filedata = filedata.replace('substitute1', "MSBR_cases")
        # Write the file out again
        with open('./MSBR_beta_{0}/MSBR_{0}.pbs'.format(eff_strings[i]), 'w', newline='\n') as file:
            file.write(filedata)
            file.close()
        file.close()

In [10]:
# MSRE
# always remember, use files in a with statement
for i in range(0, len(eff_strings)):
    with open("scale_beta_unix.pbs", "r") as myfile:
        head = list(islice(myfile, 23))
        
    with open("./MSRE_beta_{0}/MSRE_{0}.pbs".format(eff_strings[i]), "w", newline='\n') as f2:
        for item in head:
            f2.write(item)
        f2.write("cd /home/ybb/SCALE_beta/MSRE/MSRE_beta_{}/\n".format(eff_strings[i]))
        f2.write("/scale/release/6.3b4/linux/bin/scalerte MSRE_beta_{0}.inp\n".format(eff_strings[i]))
        f2.write("\n")
        
        f2.close()
    myfile.close()
        
for i in range(0, len(eff_strings)):
    with open('./MSRE_beta_{0}/MSRE_{0}.pbs'.format(eff_strings[i]), 'r') as file :
        filedata = file.read()
        # Replace the target string
        filedata = filedata.replace('substitute1', "MSRE_cases")

        # Write the file out again
        with open('./MSRE_beta_{0}/MSRE_{0}.pbs'.format(eff_strings[i]), 'w', newline='\n') as file:
            file.write(filedata)
            file.close()
        file.close()

In [11]:
# MSDR
#always remember, use files in a with statement
for i in range(0, len(eff_strings)):
    with open("scale_beta_unix.pbs", "r") as myfile:
        head = list(islice(myfile, 23))
        
    with open("./MSDR_beta_{0}/MSDR_{0}.pbs".format(eff_strings[i]), "w", newline='\n') as f2:
        for item in head:
            f2.write(item)
        f2.write("cd /home/ybb/SCALE_beta/MSDR/MSDR_beta_{}/\n".format(eff_strings[i]))
        f2.write("/scale/release/6.3b4/linux/bin/scalerte MSDR_beta_{0}.inp\n".format(eff_strings[i]))
        f2.write("\n")
        
        f2.close()
    myfile.close()
        
for i in range(0, len(eff_strings)):
    with open('./MSDR_beta_{0}/MSDR_{0}.pbs'.format(eff_strings[i]), 'r') as file :
        filedata = file.read()
        # Replace the target string
        filedata = filedata.replace('substitute1', "MSDR_cases")

        # Write the file out again
        with open('./MSDR_beta_{0}/MSDR_{0}.pbs'.format(eff_strings[i]), 'w', newline='\n') as file:
            file.write(filedata)
            file.close()
        file.close()

In [12]:
%%bash
mkdir MSBR
mkdir MSDR
mkdir MSRE
mv MSBR_beta_* ./MSBR/
mv MSDR_beta_* ./MSDR/
mv MSRE_beta_* ./MSRE/

In [13]:
# with open("scale_beta_unix.pbs", "r") as myfile:
#     head = list(islice(myfile, 23))

# always remember, use files in a with statement
with open("scale_beta.pbs", "w", newline='\n') as f2:
#     for item in head:
#         f2.write(item)
        
    # MSBR
    for i in range(0, len(eff_strings)):
        f2.write("cd /home/ybb/SCALE_beta/MSBR/MSBR_beta_{}/\n".format(eff_strings[i]))
        f2.write("qsub MSBR_{}.pbs\n".format(eff_strings[i]))
        f2.write("\n")
        
    # MSRE
    for i in range(0, len(eff_strings)):
        f2.write("cd /home/ybb/SCALE_beta/MSRE/MSRE_beta_{}/\n".format(eff_strings[i]))
        f2.write("qsub MSRE_{}.pbs\n".format(eff_strings[i]))
        f2.write("\n")
        
    # MSDR
    for i in range(0, len(eff_strings)):
        f2.write("cd /home/ybb/SCALE_beta/MSDR/MSDR_beta_{}/\n".format(eff_strings[i]))
        f2.write("qsub MSDR_{}.pbs\n".format(eff_strings[i]))
        f2.write("\n")
    f2.close()

In [31]:
# %%bash
# mkdir NewInputs
# mv MSBR_inputs ./MSBR_inputs/
# mv MSDR_inputs ./MSDR_inputs/
# mv MSRE_inputs ./MSRE_inputs/

In [32]:
# with open('./MSBR_cases.pbs', 'r') as file :
#     filedata = file.read()
#     # Replace the target string
#     filedata = filedata.replace('substitute1', "Allocating")
    
#     # Write the file out again
#     with open('MSBR_cases.pbs', 'w', newline='\n') as file:
#         file.write(filedata)
#         file.close()
#     file.close()

In [18]:
# with open("scale_beta_unix.pbs", "r") as myfile:
#     head = list(islice(myfile, 23))

# # always remember, use files in a with statement
# with open("MSBR_cases.pbs", "w", newline='\n') as f2:
#     for item in head:
#         f2.write(item)
#     for i in range(0, len(eff_strings)):
#         f2.write("cd /home/ybb/SCALE_beta/MSBR_inputs/MSBR_beta_{}/\n".format(eff_strings[i]))
#         f2.write("/scale/release/6.3b4/linux/bin/scalerte MSBR_beta_{}.inp\n".format(eff_strings[i]))
#         f2.write("\n")
#     f2.close()
    
# with open('./MSBR_cases.pbs', 'r') as file :
#     filedata = file.read()
#     # Replace the target string
#     filedata = filedata.replace('substitute1', "MSBR_cases")
    
#     # Write the file out again
#     with open('MSBR_cases.pbs', 'w', newline='\n') as file:
#         file.write(filedata)
#         file.close()
#     file.close()

In [15]:
# with open("scale_beta_unix.pbs", "r") as myfile:
#     head = list(islice(myfile, 23))

# # always remember, use files in a with statement
# with open("MSDR_cases.pbs", "w", newline='\n') as f2:
#     for item in head:
#         f2.write(item)
#     for i in range(0, len(eff_strings)):
#         f2.write("cd /home/ybb/SCALE_beta/MSDR_inputs/MSDR_beta_{}/\n".format(eff_strings[i]))
#         f2.write("/scale/release/6.3b4/linux/bin/scalerte MSDR_beta_{}.inp\n".format(eff_strings[i]))
#         f2.write("\n")
#     f2.close()
    
# with open('./MSDR_cases.pbs', 'r') as file :
#     filedata = file.read()
#     # Replace the target string
#     filedata = filedata.replace('substitute1', "MSDR_cases")
    
#     # Write the file out again
#     with open('MSDR_cases.pbs', 'w', newline='\n') as file:
#         file.write(filedata)
#         file.close()
#     file.close()

In [16]:
# with open("scale_beta_unix.pbs", "r") as myfile:
#     head = list(islice(myfile, 23))

# # always remember, use files in a with statement
# with open("MSRE_cases.pbs", "w", newline='\n') as f2:
#     for item in head:
#         f2.write(item)
#     for i in range(0, len(eff_strings)):
#         f2.write("cd /home/ybb/SCALE_beta/MSRE_inputs/MSRE_beta_{}/\n".format(eff_strings[i]))
#         f2.write("/scale/release/6.3b4/linux/bin/scalerte MSRE_beta_{}.inp\n".format(eff_strings[i]))
#         f2.write("\n")
#     f2.close()
    
# with open('./MSRE_cases.pbs', 'r') as file :
#     filedata = file.read()
#     # Replace the target string
#     filedata = filedata.replace('substitute1', "MSRE_cases")
    
#     # Write the file out again
#     with open('MSRE_cases.pbs', 'w', newline='\n') as file:
#         file.write(filedata)
#         file.close()
#     file.close()

In [17]:
# # replacement strings
# WINDOWS_LINE_ENDING = b'\r\n'
# UNIX_LINE_ENDING = b'\n'

# # relative or absolute file path, e.g.:
# file_path = r"./MSRE_cases.pbs"

# with open(file_path, 'rb') as open_file:
#     content = open_file.read()

# content = content.replace(WINDOWS_LINE_ENDING, UNIX_LINE_ENDING)

# with open(file_path, 'wb') as open_file:
#     open_file.write(content)

In [56]:
# %%bash
# mv MSBR*.pbs ./MSBR_inputs/
# mv MSDR*.pbs ./MSDR_inputs/
# mv MSRE*.pbs ./MSRE_inputs/